## Lab 04

## Zadanie 1 (Igor Sieradzki)

Funkcja <t>flow_rate</i> do modyfikacji

In [2]:
def flow_rate(weight, time, *, period: int = 60, units_per_kg: int = 1000):
    """ Funkcja wylicza ile wagi produktu przybyło/ubyło w jednostce czasu """
    return weight / time

weight = 0.5
time = 3

In [3]:
flow = flow_rate(weight, time)
print("{0:.3} kg per second".format(flow))

0.167 kg per second


In [5]:
# Zmodyfikuj funckje <i>flow_rate</i> tak, aby poniższe wywołania działały poprawnie.

def flow_rate(weight, time, *, period: int = 60, units_per_kg: int = 1000):
    """ Funkcja wylicza ile wagi produktu przybyło/ubyło w jednostce czasu """
    return (weight / units_per_kg) / (time / period)

weight = 0.5
time = 3

In [6]:
flow = flow_rate(weight, time, period=60, units_per_kg=1000)
print("{} grams per minute".format(flow))

0.01 grams per minute


In [7]:
flow = flow_rate(weight, time, period=1, units_per_kg=1)
print("{0:.3} kg per second".format(flow))

0.167 kg per second


In [8]:
flow = flow_rate(weight, time)
print("{0:.3} grams per minute".format(flow))

0.01 grams per minute


Wyjaśnić czemu poniższe rzucanie błędu jest porządane

In [9]:
try:
    flow = flow_rate(weight, time, 3600, 2.2)
except TypeError:
    print(True)

True


## Zadanie 2 (Igor Sieradzki)

Dopisz definicje dekoratora _timeit_, który wypisze na wyjscie standardowe czas wywołania udekorowanej funckji np.

```
@timeit
def foo(x):
    return x**2
    
r = foo(2)
```
wypize: <br>
    `Function foo took: 0.00001 seconds` <br>
oraz zwróci 4

In [15]:
from time import time

def timeit(func):
    """ Wypisuje czas wywołania udekorowanej funckji """

    def calculate_time(*args, **kwargs):
        begin = time()
        value = func(*args, **kwargs)
        end = time()
        print(f"Function {func.__name__} took: {end - begin} seconds")
        return value

    return calculate_time

@timeit
def squares_list(n):
    squares = []
    for i in range(n):
        squares.append(i ** 2)
    return squares

@timeit
def squares_comprehension(n):
    return [i ** 2 for i in range(n)]

@timeit
def squares_map(n):
    return map(lambda x: x**2, range(n))

n = 1000000
l = squares_list(n)
c = squares_comprehension(n)
m = squares_map(n)

Function squares_list took: 0.8745365142822266 seconds
Function squares_comprehension took: 0.5513730049133301 seconds
Function squares_map took: 0.0 seconds


## Zadanie 3 (Igor Sieradzki)

Dopisz definicje dekoratora _derivate_ wg. instrukcji w _docstringu_

In [1]:
import sys
sys.float_info.epsilon # epsilon maszynowy


def derivate(epsilon=None):
    """
    Zwraca pochodną funkcji w punkcie, wg. wzoru f'(x) = [f(x+h) - f(x)]/h,
    gdzie h jest parametrem dekoratora, jeśli nie zostanie podany, należy przyjąć 1000 * epsilon maszynowy
    """

    # works like that
    # @derivative(0.01)
    # def f(x):             is  derivative(0.01)(f)
    #
    # so we need another inner function accepting function as an argument

    if epsilon is None:
        epsilon = 1000 * sys.float_info.epsilon

    def actual_decorator(func):
        def derivative_calculation(x, **kwargs):
            return (func(x + epsilon) - func(x)) / epsilon
        return derivative_calculation
    return actual_decorator


@derivate(0.01)
def f(x):
    return x*x

@derivate(0.00001)
def g(x):
    return x*x*x+3

def test(a, b, eps=1):
    return abs(round(a)-round(b)) < eps

print(test(f(100), 200.0))
print(round(f(0)) == 0.0)

print(test(g(100), 30000.0))
print(round(g(0)) == 0.0)

from random import random
for x in [random()*1000. for _ in range(20)]:
    print(f(x), 2*x, '\t', test(f(x), 2*x))
    print(g(x), 3*x**2, '\t', test(g(x), 3*x**2))

True
True
True
True
297.8615417272522 297.8515417274801 	 True
66536.66021302342 66536.6556820776 	 True
1650.2471310435794 1650.237131045944 	 True
2042461.9555473325 2042461.9415120608 	 True
393.8322166679427 393.8222166678933 	 True
116321.95971906184 116321.95375590982 	 True
932.4302997760242 932.4202997768551 	 True
652055.723965168 652055.7115769703 	 True
95.53309406196604 95.52309406198978 	 True
6843.4975619311435 6843.496124381811 	 True
844.6985532355029 844.6885532351303 	 True
535124.0769028664 535124.0639748431 	 True
1331.8108473496977 1331.8008473549216 	 True
1330270.1354026794 1330270.1227614656 	 True
1590.203465882223 1590.1934658854261 	 True
1896536.463499069 1896536.444208528 	 True
1845.3704556450248 1845.3604556450452 	 True
2554016.423225403 2554016.4084438668 	 True
4.751988744216984 4.741988744217052 	 True
16.864914067937775 16.864842937710907 	 True
735.5902493873145 735.5802493877779 	 True
405808.73712897295 405808.72746703925 	 True
1057.9552656330634

## Zadanie 4 (Igor Sieradzki)

Dopisz definicje dekoratora _accepts_ wg. instrukcji w _docstringu_

In [26]:
def accepts(*types):
    """Sprawdza czy udekorowanej funckji zostały podane odpowiednie parametry zdefiniowane 
       w argumentach dekoratora"""

    def actual_decorator(fun):
        def actual_fun(*args, **kwargs):
            arguments = list(args)
            arguments.extend(list(kwargs.values()))
            for type, arg in zip(types, arguments):
                if not isinstance(arg, type):
                    raise TypeError
            return fun(*args, **kwargs)
        return actual_fun
    return actual_decorator


@accepts(str)
def capitalize(word):
    return word[0].upper() + word[1:]

print(capitalize('ola') == 'Ola')

try:
    capitalize(2)
except TypeError:
    print(True)

@accepts(float, int)
def static_pow(base, exp):
    return base ** exp 

print(static_pow(2., 2) == 4.)
print(static_pow(2., exp=2) == 4.)
print(static_pow(base=2., exp=2) == 4.)

try:
    static_pow('x', 10)
except TypeError:
    print(True)
    
try:
    static_pow(2, 2.2)
except TypeError:
    print(True)

True
True
True
True
True
True
True


## Zadanie 5 (Igor Sieradzki)

Dopisz definicje dekoratora _returns_ wg. instrukcji w _docstringu_

In [28]:
def returns(*types):
    """Sprawdza czy udekorowana funkcja zwraca poprawne argumenty, zdefiniowane w parametrach dekoratora"""
    def actual_decorator(fun):
        def actual_fun(*args, **kwargs):
            value = fun(*args, **kwargs)
            for type, returned_value in zip(types, list(value)):
                if not isinstance(returned_value, type):
                    raise TypeError
            return value
        return actual_fun
    return actual_decorator


@returns(str)
def str_only_identity(word):
    return word

print(str_only_identity('hello') == 'hello')

try:
    str_only_identity(10)
except TypeError:
    print(True)
    
@returns(int, int)
def split_indices(x):
    return x[0], x[1]

print(split_indices(x=[6,9]) == (6,9))

try:
    split_indices('AB')
except TypeError:
    print(True)

True
True
True
True


## Zadanie 6 (Igor Sieradzki)
Stwórz dekorator cached służący do cachowania wywołań dowolnej funkcji, tzn. chcemy by:
* wywołanie funkcji z określonymi argumentami miało miejsce tylko raz
* funkcja mogła przyjmować dowolną liczbę nazwanych i nienazwanych argumentów
* nie musi reagować poprawnie na domyślne argumenty, tzn. wywołanie funkcji z domyślnymi argumentami a podanie dokładnie takich samych może być traktowane jako dwa różne wywołania
* na opakowanej funkcji można wywołać `.cache_reset()`, który usunie cache z pamięci
* wywołanie `.cache_status()` zwraca string z opisem w postaci: <br>
    `Function FUNCTION_NAME called X times, evaluated Y times`

In [ ]:
from random import random

def cached(fun):
    cache = {}
    cached.called_counter = 0
    cached.evaluated_counter = 0
    def actual_fun(*args, **kwargs):
        parsed_arguments = []
        parsed_arguments.extend(list(args))
        parsed_arguments.extend(list(kwargs.items()))
        if parsed_arguments not in cache:
            cache[parsed_arguments] = 1
            cached.evaluated_counter += 1
        else:
            cache[parsed_arguments] += 1
        cached.called_counter += 1

        return fun(*args, **kwargs)

    def cache_status():
        return f"Function {fun.__name__} called {call} times, evaluated {} times"

    def cache_reset():
        pass

    return actual_fun

@cached
def foo(x, y=1, z=4):
    return random()
        
print(foo(3) == foo(3))
print(foo(4) == foo(4))
print(foo(3, z=-1, y=3) == foo(3, y=3, z=-1))
print(foo(3) != foo(x=3))
a = foo(3)
foo.cache_reset()
print(a != foo(3))
print(foo.cache_status() == 'Function foo called 10 times, evaluated 5 times')

In [32]:
def test(*args, **kwargs):
    print(list(kwargs.items()))

test(z=3, d=4)

[('z', 3), ('d', 4)]


## Zadanie 7 (Krzysztof Hajto)

Napisz dekorator który będzie robić n-krotne złożenie funkcji, gdzie n jest parametrem dekoratora

In [29]:
def zlozenie(n):
    def actual_decorator(fun):
        def actual_fun(*args, **kwargs):
            value = fun(*args, **kwargs)
            for i in range(n-1):
                value = fun(value)
            return value
        return actual_fun
    return actual_decorator


@zlozenie(3)
def f1(x):
    return x+1

@zlozenie(2)
def f2(x):
    return x*x

@zlozenie(5)
def f3(word):
    return "".join(chr(ord(l)+1) for l in word)

print(f1(2)==5)
print(f2(3)==81)
print(f3("alamakota")=="fqfrfptyf")

True
True
True


## Zadanie 8 (Krzysztof Hajto)

Python nie ma wbudowanej instrukcji switch. Ale posiada anonimowe funkcje oraz słowniki. Zaimplementuj poniższy switch w postaci słownika funkcji.
`
int my_function(x, y) {


    switch(x) {
        case 1: return y*y;
        case 2: return x+y;
        case 3: return x*y;
        case 4: return 0;
    }
}
`

PS. Nigdy nie róbcie tego w faktycznym kodzie :)

In [30]:
def my_function(x, y):
    def one():
        return y * y
    def two():
        return x + y
    def three():
        return x * y
    def four():
        return 0

    if x == 1:
        return one()
    if x == 2:
        return two()
    if x == 3:
        return three()
    if x == 4:
        return four()
    raise ValueError

print(my_function(1,3)==9)
print(my_function(2,4)==6)
print(my_function(3,1)==3)
print(my_function(4,9)==0)

True
True
True
True
